# 0.0 Imports

In [14]:
import pandas as pd
import inflection

## 0.1 Helper Functions

## 0.2 Loading Data

In [3]:
df_sales_raw = pd.read_csv('C:\\Users\\felip\\repos\\data_science_em_producao\\rossmann_data\\train.csv', low_memory = False)
df_store_raw = pd.read_csv('C:\\Users\\felip\\repos\\data_science_em_producao\\rossmann_data\\store.csv', low_memory = False)

### [Merge - join two DataFrames.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

In [5]:
df_raw = pd.merge(df_sales_raw, df_store_raw, how = 'left', on = 'Store')

# 1.0 Data Description

## 1.1 Rename Columns

In [8]:
# Dataset copy
df1 = df_raw.copy()

In [12]:
cols_old = list(df1.columns)

In [15]:
# Transforming columns names from CamelCase to SnakeCase.

snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase, cols_old))

# Rename columns
df1.columns = cols_new

In [16]:
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

## 1.2 Data Dimensions

In [20]:
print(f'Number of Rows: {df1.shape[0]}.\nNumber of Columns: {df1.shape[1]}.')

Number of Rows: 1017209.
Number of Columns: 18.


## 1.3 Data Types

In [21]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [22]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4 Check NaN

In [24]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64